Kaggleのメルカリ価格予想チャレンジのデータを使って

https://www.kaggle.com/

https://www.kaggle.com/c/mercari-price-suggestion-challenge/data

適性な販売価格を予測していきます。

In [1]:
#インポート
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.model_selection import train_test_split
pd.set_option('display.float_format', lambda x:'%.5f' % x)
import numpy as np

In [8]:
#Googleドライブからの読み込みそしてtsvファイルからPandas DataFrameへ読み込み
def data_set():
  types_dict_train = {'train_id':'int64', 'item_condition_id':'int8', 'price':'float64', 'shipping':'int8'}
  types_dict_test = {'test_id':'int64', 'item_condition_id':'int8', 'shipping':'int8'}
  train = pd.read_csv("/content/drive/My Drive/基礎集計入門/mercari/train.tsv", delimiter='\t', low_memory=True, dtype=types_dict_train)
  test = pd.read_csv("/content/drive/My Drive/基礎集計入門/mercari/test.tsv", delimiter='\t', low_memory=True, dtype=types_dict_test)


trainとtestのデータフレームの冒頭5行を表示させる

In [11]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.00000,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.00000,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.00000,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.00000,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.00000,0,Complete with certificate of authenticity


In [13]:
test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


train_id / test _id – ユーザー投稿のID

name – 投稿のタイトル。タイトルに価格に関する情報がある場合（例：$20）はメルカリが事前に削除をして[rm]と置き換えている

item_condition_id – ユーザーが指定した商品の状態

category_name – 投稿カテゴリー

brand_name – ブランドの名前

price – 訓練データのみ。実際に売られた価格。米ドル表示。

shipping – 送料のフラグ。「1」は販売者負担。「0」は購入者負担。

item_description – ユーザーが投稿した商品説明の全文。価格情報がある場合は[rm]と置き換えられている。

testのpriceを予測します。

In [14]:
# trainとtestのサイズを確認
train.shape, test.shape

((1482535, 8), (693359, 7))

In [16]:
def display_all(df):
  with pd.option_context("display.max_rows", 1000):
    with pd.option_context("display.max_columns", 1000):
      display(df)
# trainの基本統計量を表示
display_all(train.describe(include='all').transpose())


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
train_id,1482535.00000,NaN,NaN,NaN,741267.00000,427971.13500,0.00000,370633.50000,741267.00000,1111900.50000,1482534.00000
name,1482535,1225273,Bundle,2232,NaN,NaN,NaN,NaN,NaN,NaN,NaN
item_condition_id,1482535.00000,NaN,NaN,NaN,1.90738,0.90316,1.00000,1.00000,2.00000,3.00000,5.00000
category_name,1476208,1287,"Women/Athletic Apparel/Pants, Tights, Leggings",60177,NaN,NaN,NaN,NaN,NaN,NaN,NaN
brand_name,849853,4809,PINK,54088,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price,1482535.00000,NaN,NaN,NaN,26.73752,38.58607,0.00000,10.00000,17.00000,29.00000,2009.00000
shipping,1482535.00000,NaN,NaN,NaN,0.44727,0.49721,0.00000,0.00000,0.00000,1.00000,1.00000
item_description,1482531,1281426,No description yet,82489,NaN,NaN,NaN,NaN,NaN,NaN,NaN


item_description」の「Top」では「No description yet」と出ており商品説明がしっかり入っている訳ではない。価格（price）は平均（mean）が26.74ドルで標準偏差（std)が38.59ドル。最小値が0ドル、最大値は2009ドルとなっている。

ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

In [20]:
# trainのカテゴリ名、商品説明、投稿タイトル、ブランド名のデータタイプを「category」へ変換する
train.category_name = train.category_name.astype('category')
train.item_description = train.item_description.astype('category')
train.name = train.name.astype('category')
train.brand_name = train.brand_name.astype('category')
# testのカテゴリ名、商品説明、投稿タイトル、ブランド名のデータタイプを「category」へ変換する
test.category_name = test.category_name.astype('category')
test.item_description = test.item_description.astype('category')
test.name = test.name.astype('category')
test.brand_name = test.brand_name.astype('category')
# dtypesでデータ形式の確認
train.dtypes,test.dtypes


(train_id                int64
 name                 category
 item_condition_id        int8
 category_name        category
 brand_name           category
 price                 float64
 shipping                 int8
 item_description     category
 dtype: object, test_id                 int64
 name                 category
 item_condition_id        int8
 category_name        category
 brand_name           category
 shipping                 int8
 item_description     category
 dtype: object)

データのユニーク値の確認

In [22]:
# trainの中のユニークな値を確認する
train.apply(lambda x: x.nunique())


train_id             1482535
name                 1225273
item_condition_id          5
category_name           1287
brand_name              4809
price                    828
shipping                   2
item_description     1281426
dtype: int64

In [23]:
# testの中のユニークな値を確認する
test.apply(lambda x: x.nunique())

test_id              693359
name                 601117
item_condition_id         5
category_name          1223
brand_name             3900
shipping                  2
item_description     609555
dtype: int64

カテゴリやブランド名の重複、投稿タイトル（name）の重複データが確認できる。商品説明には「No Description yet」（商品説明はまだない）があったので、重複が確認できる。

ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

欠損データの確認

In [24]:
# trainの欠損データの個数と%を確認
train.isnull().sum(),train.isnull().sum()/train.shape[0]

(train_id                  0
 name                      0
 item_condition_id         0
 category_name          6327
 brand_name           632682
 price                     0
 shipping                  0
 item_description          4
 dtype: int64, train_id            0.00000
 name                0.00000
 item_condition_id   0.00000
 category_name       0.00427
 brand_name          0.42676
 price               0.00000
 shipping            0.00000
 item_description    0.00000
 dtype: float64)

In [25]:
# testの欠損データの個数と%を確認
test.isnull().sum(),test.isnull().sum()/test.shape[0]

(test_id                   0
 name                      0
 item_condition_id         0
 category_name          3058
 brand_name           295525
 shipping                  0
 item_description          0
 dtype: int64, test_id             0.00000
 name                0.00000
 item_condition_id   0.00000
 category_name       0.00441
 brand_name          0.42622
 shipping            0.00000
 item_description    0.00000
 dtype: float64)

カテゴリ名（category_name）とブランド名（brand_name）は欠損数が大きい。特にブランド名の欠損度合いはかなり大きい。

ーーーーーーーーーーーーーーーーーーーーーーーーーーーー

ランダムフォレストのモデルを作成するためのデータの事前処理

trainとtestのIDのカラム名を変更し、データ連結後でもどちらのデータに属しているかわかるようにフラグをつける

In [26]:
# trainとtestのidカラム名を変更する
train = train.rename(columns = {'train_id':'id'})
test = test.rename(columns = {'test_id':'id'})
# 両方のセットへ「is_train」のカラムを追加
# 1 = trainのデータ、0 = testデータ
train['is_train'] = 1
test['is_train'] = 0
# trainのprice(価格）以外のデータをtestと連結
train_test_combine = pd.concat([train.drop(['price'], axis=1),test],axis=0)
# データの中身の表示
train_test_combine.head()

,id,name,item_condition_id,category_name,brand_name,shipping,item_description,is_train
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,1,No description yet,1
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,0,This keyboard is in great condition and works ...,1
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,1,Adorable top with a hint of lace and a key hol...,1
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,1,New with tags. Leather horses. Retail for [rm]...,1
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,0,Complete with certificate of authenticity,1


In [27]:
#shapeの確認
train_test_combine.shape

(2175894, 8)

「(2175894, 8)」が訓練データ1,482,535個とテストデータ693,359個の合計になっていることが確認できる

ーーーーーーーーーーーーーーーーーーーーーーーー

連結させたデータ（train_test_combine）のデータ形式を変更

In [29]:
# train_test_combineの文字列のデータタイプを「category」へ変換
train_test_combine.category_name = train_test_combine.category_name.astype('category')
train_test_combine.item_description = train_test_combine.item_description.astype('category')
train_test_combine.name = train_test_combine.name.astype('category')
train_test_combine.brand_name = train_test_combine.brand_name.astype('category')
# combinedDataの文字列を「.cat.codes」で数値へ変換する
train_test_combine.name = train_test_combine.name.cat.codes
train_test_combine.category_name = train_test_combine.category_name.cat.codes
train_test_combine.brand_name = train_test_combine.brand_name.cat.codes
train_test_combine.item_description = train_test_combine.item_description.cat.codes
# データの中身とデータ形式の確認
train_test_combine.head()

,id,name,item_condition_id,category_name,brand_name,shipping,item_description,is_train
0,0,916335,3,830,0,1,1172054,1
1,1,1292428,3,87,3890,0,1585540,1
2,2,131013,1,1278,4589,1,167134,1
3,3,802671,1,504,0,1,1136644,1
4,4,65051,1,1205,0,0,531910,1


In [30]:
train_test_combine.dtypes

id                   int64
name                 int32
item_condition_id     int8
category_name        int16
brand_name           int16
shipping              int8
item_description     int32
is_train             int64
dtype: object

数値に変換したので、またtestとtrainに分ける

In [32]:
# 「is_train」のフラグでcombineからtestとtrainへ切り分ける
df_test = train_test_combine.loc[train_test_combine['is_train'] == 0]
df_train = train_test_combine.loc[train_test_combine['is_train'] == 1]
# 「is_train」をtrainとtestのデータフレームから落とす
df_test = df_test.drop(['is_train'], axis=1)
df_train = df_train.drop(['is_train'], axis=1)
# サイズの確認をしておきましょう
df_test.shape, df_train.shape

((693359, 7), (1482535, 7))

df_train」（訓練データ）へ「train」の価格（price）を戻して、log関数で処理をする

In [38]:
# df_trainへprice（価格）を戻す
df_train['price'] = train.price
# price（価格）をlog関数で処理
df_train['price'] = df_train['price'].apply(lambda x: np.log(x) if x>0 else x)
# df_trainを表示して確認
df_train.head()

,id,name,item_condition_id,category_name,brand_name,shipping,item_description,price
0,0,916335,3,830,0,1,1172054,2.30259
1,1,1292428,3,87,3890,0,1585540,3.95124
2,2,131013,1,1278,4589,1,167134,2.30259
3,3,802671,1,504,0,1,1136644,3.55535
4,4,65051,1,1205,0,0,531910,3.78419


ランダムフォレストのモデル作成

In [40]:
# x ＝ price以外の全ての値、y = price（ターゲット）で切り分ける
x_train, y_train = df_train.drop(['price'], axis=1), df_train.price
# モデルの作成
m = RandomForestRegressor(n_jobs=-1, min_samples_leaf=5, n_estimators=200)
m.fit(x_train, y_train)
# スコアを表示
m.score(x_train, y_train)

0.7400987967569919

スコアは0.7400987967569919となった

モデルが作れたので、先に処理をした「df_test」（テスト用データ）を入れて、実際の予測値を出す

In [36]:
# 作成したランダムフォレストのモデル「m」に「df_test」を入れて予測する
preds = m.predict(df_test)
# 予測値 predsをnp.exp()で処理
np.exp(preds)
# Numpy配列からpandasシリーズへ変換
preds = pd.Series(np.exp(preds))
# テストデータのIDと予測値を連結
submit = pd.concat([df_test.id, preds], axis=1)
# カラム名の名前をつける
submit.columns = ['test_id', 'price']
submit.head()

,test_id,price
0,0,12.88961
1,1,7.75365
2,2,19.99445
3,3,14.85366
4,4,8.02702


結論
調べながらやってみたが正直理解できていない。スコアはもう少し上げていけると思う